<a href="https://www.kaggle.com/code/aleksandrmorozov123/cv-object-detection?scriptVersionId=198563926" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Crowd counting**

In [1]:
import torch
import os
import h5py
from scipy import io

if not os.path.exists('CSRNet-pytorch/'):
    %pip install -U scipy torch_snippets torch_summary
    !git clone https://github.com/sizhky/CSRNet-pytorch.git

%cd CSRNet-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.6/204.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 69.9 MB/s eta 0:00:00:00:01
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26306 sha256=fd2335c806b56a3b02cdcd8b6c68391739f9b8bf66f46733cc48c056d2fdb60e
  Stored in directory: /root/.cache/pip/wheels/7c/d0/9e/1f26ebb66d9e1732e4098bc5a6c2d91f6c9a529838f0284890
Successfully built typing
Note: you may need to restart the kernel to use updated packages.
Cloning into 'CSRNet-pytorch'...
remote: Enumer

In [29]:
from torch_snippets import *
from torchvision import transforms
from torch import optim
from torch_snippets.torch_loader import Report

In [31]:
part_A = '/kaggle/input/shanghaitech-with-people-density-map/ShanghaiTech/part_A/train_data/'
image_folder = '/kaggle/input/shanghaitech-with-people-density-map/ShanghaiTech/part_A/train_data/images/'

heatmap_folder = '/kaggle/input/shanghaitech-with-people-density-map/ShanghaiTech/part_A/train_data/ground-truth-h5/'
gt_folder = '/kaggle/input/shanghaitech-with-people-density-map/ShanghaiTech/part_A/train_data/ground-truth/'

In [8]:
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

class MyDataset(Dataset):
    def __init__(self,x,y):
        self.x = torch.tensor(x).float()
        self.y = torch.tensor(y).float()
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]

In [33]:
# define training and validation dataset
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tfm = transforms.Compose([transforms.ToTensor()])

class Crowds(Dataset):
    def __init__(self, stems):
        self.stems = stems

    def __len__(self):
        return len(self.stems)

    def __getitem__(self, ix):
        _stem = self.stems[ix]
        image_path = f'{image_folder}/{_stem}.jpg'
        heatmap_path = f'{heatmap_folder}/{_stem}.h5'
        gt_path = f'{gt_folder}/GT_{_stem}.mat'

        pts = io.loadmat(gt_path)
        pts = len(pts['image_info'][0,0][0,0][0])

        image = read(image_path, 1)
        with h5py.File(heatmap_path, 'r') as hf:
            gt = hf['density'][:]
        gt = resize(gt, 1/8)*64
        return image.copy(), gt.copy(), pts

    def collate_fn(self, batch):
        ims, gts, pts = list(zip(*batch))
        ims = torch.cat([tfm(im)[None] for im in ims]).to(device)
        gts = torch.cat([tfm(gt)[None] for gt in gts]).to(device)
        return ims, gts, torch.tensor(pts).to(device)

    def choose(self):
        return self[randint(len(self))]

from sklearn.model_selection import train_test_split
trn_stems, val_stems = train_test_split(stems(Glob(image_folder)), random_state=10)

trn_ds = Crowds(trn_stems)
val_ds = Crowds(val_stems)

trn_dl = DataLoader(trn_ds, batch_size=1, shuffle=True, collate_fn=trn_ds.collate_fn)
val_dl = DataLoader(val_ds, batch_size=1, shuffle=True, collate_fn=val_ds.collate_fn)

In [10]:
from torchvision import models
from utils import save_net,load_net

def make_layers(cfg, in_channels = 3,batch_norm=False,dilation = False):
    if dilation:
        d_rate = 2
    else:
        d_rate = 1
    layers = []
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=d_rate, dilation=d_rate)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)

In [21]:
# define the network architecture
class CSRNet (nn.Module):
    def __init__ (self, load_weights = False):
        super (CSRNet, self).__init__ ()
        self.seen = 0
        self.frontend_feat = [64, 64, 'M', 128, 128,
                             'M', 256, 256, 256, 'M', 512, 512, 512]
        self.backend_feat = [512, 512, 512, 256, 128, 64]
        self.frontend = make_layers (self.frontend_feat)
        self.backend = make_layers (self.backend_feat, in_channels = 512, dilation = True)
        self.output_layer = nn.Conv2d (64, 1, kernel_size = 1)
        if not load_weights: 
            mod = models.vgg16 (pretrained = True)
            self._initialize_weights ()
            items = list (self.frontend.state_dict ().items())
            _items = list (mod.state_dict ().items())
            for i in range (len (self.frontend.state_dict().items ())):
                items[i][1].data[:] = _items[i][1].data[:]
    def forward (self, x):
        x = self.frontend (x)
        x = self.backend (x)
        x = self.output_layer (x)
        return x
    def _initialize_weights (self):
        for m in self.modules ():
            if isinstance (m, nn.Conv2d):
                nn.init.normal_ (m.weight, std = 0.01)
                if m.bias is not None:
                    nn.init.constant_ (m.bias, 0)
            elif isinstance (m, nn.BatchNorm2d):
                nn.init.constant_ (m.weight, 1)
                nn.init.constant_ (m.bias, 0)
    

In [22]:
# train and validate a batch of data
def train_batch (model, data, optimizer, criterion):
    model.train ()
    optimizer.zero_grad ()
    ims, gts, pts = data
    _gts = model (ims)
    loss = criterion (_gts, gts)
    loss.backward ()
    optimizer.step ()
    pts_loss = nn.L1Loss ()(_gts.sum(), gts.sum())
    return loss.item (), pts_loss.item ()

@torch.no_grad()
def validate_batch (model, data, criterion):
    model.eval ()
    ims, gts, pts = data
    _gts = model (ims)
    loss = criterion (_gts, gts)
    pts_loss = nn.L1Loss ()(_gts.sum(), gts.sum())
    return loss.item(), pts_loss.item

In [34]:
# train the model
model = CSRNet ().to(device)
criterion = nn.MSELoss ()
optimizer = optim.Adam (model.parameters (), lr = 1e-6)
n_epochs = 20

log = Report (n_epochs)
for ex in range (n_epochs):
    N = len (trn_dl)
    for bx, data in enumerate (trn_dl):
        loss, pts_loss = train_batch (model, data, optimizer, criterion)
        log.record (ex+(bx+1)/N, trn_loss = loss, trn_pts_loss = pts_loss, end = '\r')
        N = len (val_dl)
        for bx, data in enumerate (val_dl):
            loss, pts_loss = validate_batch (model, data, criterion)
            log.record (ex+(bx+1)/N, val_loss = loss, val_pts_loss = pts_loss, end = '\r')
        log.report_avgs (ex + 1)
        if ex == 10: optimizer = optim.Adam (model.parameters (), lr = 1e-7)

AttributeError: 'function' object has no attribute 'loadmat'

In [17]:
# make the inference 
from matplotlib import cm as c
from PIL import Image
from torchvision import datasets, transforms
transform = transforms.Compose ([
    transforms.ToTensor (), transforms.Normalize (mean = [0.485, 0.456, 0.406],
                                                 std = [0.229, 0.224, 0.225]),
])
test_folder = '/kaggle/input/shanghaitech-with-people-density-map/ShanghaiTech/part_A/test_data/'
imgs = Glob (f'{test_folder}/images')
f = choose (imgs)
print (f)
img = transform (Image.open(f).convert('RGB')).to(device)

/kaggle/input/shanghaitech-with-people-density-map/ShanghaiTech/part_A/test_data/images/IMG_126.jpg


In [19]:
# pass the image through the trained model
output = model (img[None])
print ("Predicted Count: ", int (output.detach().cpu().sum().numpy()))
temp = np.asarray (output.detach ().cpu ()\
                  .reshape (output.detach().cpu()\
                           .shape[2], output.detach()\
                           .cpu().shape[3]))
plt.imshow (temp, cmap = c.jet)
plt.show ()

NameError: name 'model' is not defined